In [ ]:
# default_exp core.rq_inventory_upload

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *
import json
import responses
from pathlib import Path
from discogspy.core import *
import pandas as pd
from pathlib import Path

In [ ]:
#hide
with open(Path("../config/example_user_info.json"), "r") as user_config_file:
    config = json.load(user_config_file)
    example_user = UserWithUserTokenBasedAuthentication(user_token=config["user_token"],
                                                        user_agent=config["user_agent"])

# Core Inventory Upload Requests

> Core.rq_inventory_upload is a collection of function wrappers around the Discogs API for inventory uploads.

In [ ]:
#export
import requests
from typing import Union
import io
from discogspy.core import *

In [ ]:
#export


def add_new_listings_using_csv(user: UserWithUserTokenBasedAuthentication,
                               file: io.BufferedReader
                               ) -> requests.models.Response:
    """
    Upload a csv file of listings to add to the given user inventory.
    
    Note: The file you upload must be a comma-separated CSV. 
          The first row must be a header with lower case field names.
          All listings will be marked “For Sale” immediately. 
          Currency information will be pulled from your marketplace settings. 
          Any fields that aren’t optional or required will be ignored.
          
    User Authentication needed. 
    
    Parameters:
    
    user: user object (required)
    
    file: file (required) 
        -> The CSV file of items to add to your inventory.
        
    Csv File Values:

    release_id: number (required)
        -> This value corresponds with the Discogs database Release ID.
        
    media_condition: string (required)
        -> The condition of the release you are posting.
        
    price: number (required)
        -> The price of the item (in the seller’s currency).

    status: string (required)
        -> The status of the listing. Defaults to For Sale. 
    
    sleeve_condition: string (optional)
        -> The condition of the sleeve of the item you are posting.     
    
    comments: string (optional) 
        -> Any remarks about the item that will be displayed to buyers.
        
    allow_offers: boolean (optional)
        -> Must be Y or N.
               
    external_id: string (optional) 
        -> A freeform field that can be used for the seller’s own reference. 
        -> Information stored here will not be displayed to anyone other than the seller. 
        -> This field is called “Private Comments” on the Discogs website.
    
    location: string (optional) 
        -> A freeform field that is intended to help identify an item’s physical storage location. 
        -> Information stored here will not be displayed to anyone other than the seller. 
        -> This field will be visible on the inventory management page and will be available in inventory exports via the website.
    
    weight: number (optional)
        -> The weight, in grams, of this listing, for the purpose of calculating shipping. 
        -> Set this field to auto to have the weight automatically estimated for you.
    
    format_quantity: number (optional)
        -> The number of items this listing counts as, for the purpose of calculating shipping. 
        -> This field is called “Counts As” on the Discogs website. 
        -> Set this field to auto to have the quantity automatically estimated for you.
    """
    
    url = f"{INVENTORY_UPLOAD_URL}/add"
    headers = user.headers
    params = user.params
    files = {"upload": file}

    return requests.post(url, headers=headers, params=params, files=files)

In [ ]:
# Example for usage of add_new_listings_using_csv

csv_path = Path("../test_example/file.csv")

with open(csv_path, "rb") as upload_file:
    add_new_listings_using_csv(user=example_user, 
                               file=upload_file)
    
pd.read_csv(csv_path)

,release_id,price,media_condition,comments
0,123,1.5,Mint (M),Comments about this release for sale
1,456,2.5,Near Mint (NM or M-),More comments
2,7890,3.5,Good (G),Signed vinyl copy


In [ ]:
#hide

@responses.activate
def test_add_new_listings_using_csv___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{INVENTORY_UPLOAD_URL}/add?token=test_user_token"
    
    result_content = {"TestUpload": "TestContent"}
    
    responses.add(responses.POST, target_url,
                  json=result_content, status=200)
    
    with open("../test_example/file.csv", "rb") as test_file:
        resp = add_new_listings_using_csv(user=test_user, file=test_file)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_add_new_listings_using_csv___authentication_user_token()

In [ ]:
#export


def update_existing_listings_using_csv(user: UserWithUserTokenBasedAuthentication,
                                       file: io.BufferedReader
                                       ) -> requests.models.Response:
    """
    Upload a CSV file with updated information for existing listings in given user inventory.
    
    Note: The file you upload must be a comma-separated CSV. 
          The first row must be a header with lower case field names.
          All listings will be marked “For Sale” immediately. 
          Currency information will be pulled from your marketplace settings. 
          Any fields that aren’t optional or required will be ignored.
    
    User Authentication needed. 
    
    Parameters:
    
    user: user object (required)
    
    file: file (required) 
        -> The CSV file of items to add to your inventory.
        
    Csv File Values:

    release_id: number (optional)
        -> This value corresponds with the Discogs database Release ID.
        
    listing_id: number (required)
        -> This is the ID of the listing you wish to update.
        
    media_condition: string (optional)
        -> The condition of the release you are posting.
        
    price: number (optional)
        -> The price of the item (in the seller’s currency).

    status: string (optional)
        -> The status of the listing. Defaults to For Sale. 
    
    sleeve_condition: string (optional)
        -> The condition of the sleeve of the item you are posting.     
    
    comments: string (optional) 
        -> Any remarks about the item that will be displayed to buyers.
        
    allow_offers: boolean (optional)
        -> Must be Y or N.
               
    external_id: string (optional) 
        -> A freeform field that can be used for the seller’s own reference. 
        -> Information stored here will not be displayed to anyone other than the seller. 
        -> This field is called “Private Comments” on the Discogs website.
    
    location: string (optional) 
        -> A freeform field that is intended to help identify an item’s physical storage location. 
        -> Information stored here will not be displayed to anyone other than the seller. 
        -> This field will be visible on the inventory management page and will be available in inventory exports via the website.
    
    weight: number (optional)
        -> The weight, in grams, of this listing, for the purpose of calculating shipping. 
        -> Set this field to auto to have the weight automatically estimated for you.
    
    format_quantity: number (optional)
        -> The number of items this listing counts as, for the purpose of calculating shipping. 
        -> This field is called “Counts As” on the Discogs website. 
        -> Set this field to auto to have the quantity automatically estimated for you.
    """
    
    url = f"{INVENTORY_UPLOAD_URL}/change"
    headers = user.headers
    params = user.params
    files = {"upload": file}

    return requests.post(url, headers=headers, params=params, files=files)

In [ ]:
# Example for usage of update_existing_listings_using_csv
    
csv_path = Path("../test_example/file.csv")

with open(csv_path, "rb") as upload_file:
    update_existing_listings_using_csv(user=example_user, 
                                       file=upload_file)
    
pd.read_csv(csv_path)
    

,release_id,listing_id,price,media_condition,comments
0,123,456,1.5,Mint (M),Comments about this release for sale
1,456,789,1.5,Near Mint (NM or M-),More comments
2,7890,101112,1.5,Good (G),Signed vinyl copy


In [ ]:
#hide

@responses.activate
def test_update_existing_listings_using_csv___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{INVENTORY_UPLOAD_URL}/change?token=test_user_token"
    
    result_content = {"TestUpload": "TestContent"}
    
    responses.add(responses.POST, target_url,
                  json=result_content, status=200)
    
    with open("../test_example/file.csv", "rb") as test_file:
        resp = update_existing_listings_using_csv(user=test_user, file=test_file)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_update_existing_listings_using_csv___authentication_user_token()

In [ ]:
#export


def delete_listings_using_csv(user: UserWithUserTokenBasedAuthentication,
                              file: io.BufferedReader
                              ) -> requests.models.Response:
    """
    Upload a CSV file with listings to delete from the given inventory.
    
    Note: The file you upload must be a comma-separated CSV. 
          The first row must be a header with lower case field names.
          
    User Authentication needed. 
    
    Parameters:
    
    user: user object (required)
    
    file: file (required) 
        -> The CSV file of items to add to your inventory.
        
    Csv File Values:

    listing_id: number (required)
        -> This is the ID of the listing you wish to delete.
    """
    
    url = f"{INVENTORY_UPLOAD_URL}/delete"
    headers = user.headers
    params = user.params
    files = {"upload": file}

    return requests.post(url, headers=headers, params=params, files=files)

In [ ]:
# Example for usage of delete_listings_using_csv
   
csv_path = Path("../test_example/delete_file.csv")

with open(csv_path, "rb") as upload_file:
    delete_listings_using_csv(user=example_user, 
                              file=upload_file)
    
pd.read_csv(csv_path)

,listing_id
0,12345678
1,98765432
2,31415926


In [ ]:
#hide

@responses.activate
def test_delete_listings_using_csv___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{INVENTORY_UPLOAD_URL}/delete?token=test_user_token"
    
    result_content = {"TestUpload": "TestContent"}
    
    responses.add(responses.POST, target_url,
                  json=result_content, status=200)
    
    with open("../test_example/delete_file.csv", "rb") as test_file:
        resp = delete_listings_using_csv(user=test_user, file=test_file)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_delete_listings_using_csv___authentication_user_token()  

In [ ]:
#export


def get_list_of_recent_uploads(user: UserWithUserTokenBasedAuthentication,
                               page: Union[int, None] = None,
                               per_page: Union[int, None] = None
                               ) -> requests.models.Response:
    """
    Get a list of all recent inventory uploads for the given inventory.
    
    User Authentication needed.
    
    Parameters:
    
    user: user object (required).
    
    page: number (optional) 
        -> The page you want to request.
        
    per_page: number (optional)
        -> The number of items per page.
    """
    
    url = f"{INVENTORY_UPLOAD_URL}"
    headers = user.headers
    params = user.params

    if page:
        params["page"] = max(1, page)
    if per_page:
        params["per_page"] = max(1, per_page)

    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_list_of_recent_uploads
get_list_of_recent_uploads(user=example_user, 
                           page=1, 
                           per_page=3)
  

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_list_of_recent_uploads___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{INVENTORY_UPLOAD_URL}?token=test_user_token&page=1&per_page=3"
    
    result_content = {"TestUpload": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)
    
    resp = get_list_of_recent_uploads(user=test_user, page=1, per_page=3)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_get_list_of_recent_uploads___authentication_user_token()  

In [ ]:
#export


def get_upload(user: UserWithUserTokenBasedAuthentication,
               upload_id: int
               ) -> requests.models.Response:
    """
    Get details about the status of the given inventory upload.
    
    User Authentication needed. 
    
    Parameters:
    
    user: user object (required).
    
    upload_id: number (required) 
        -> Id of the upload.
    """
    
    url = f"{INVENTORY_UPLOAD_URL}/{upload_id}"
    headers = user.headers
    params = user.params

    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_upload
get_upload(user=example_user, 
           upload_id=191405)

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_upload___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{INVENTORY_UPLOAD_URL}/123456?token=test_user_token"
    
    result_content = {"TestUpload": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)
    
    resp = get_upload(user=test_user, upload_id=123456)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_get_upload___authentication_user_token()  

In [ ]:
#hide

from nbdev.export import *
notebook2script()

Converted 00_core.constants.ipynb.
Converted 01_core.discogs_user.ipynb.
Converted 02_core.rq_database.ipynb.
Converted 03_core.rq_marketplace.ipynb.
Converted 04_core.rq_inventory_export.ipynb.
Converted 05_core.rq_inventory_upload.ipynb.
Converted 06_core.rq_collection.ipynb.
Converted 07_core.rq_wantlist.ipynb.
Converted index.ipynb.
